In [1]:
%pip install git+https://github.com/tinnguyen197662/keras-vggface.git
%pip install pyyaml h5py


  Cloning https://github.com/tinnguyen197662/keras-vggface.git to c:\users\dell\appdata\local\temp\pip-req-build-4z79fete
  Resolved https://github.com/tinnguyen197662/keras-vggface.git to commit bccd893f44143a8e07d0916afd1ddd2183540059
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/143.2 kB ? eta -:--:--
     ----------                            41.0/143.2 kB 653.6 kB/s eta 0:00:01
     -----------------------------------    133.1/143.2 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 143.2/143.2 kB 1.2 MB/s eta 0:00:00
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8377 sha256=d21235fa502c6e842122e302fb0e005074312f5fcb3a641280d85fcc8583c5a1
  Stored in directory: C:\Users\DELL\AppData\Local\Temp\pip-ephem-wheel-cache-ol_0k9_h\wheels\90\c5\09\f57ff006b35731c5fcc639d9b07001fb5f012e64508c921e8e
Successfully built keras-vggface

  Running command git clone --filter=blob:none --quiet https://github.com/tinnguyen197662/keras-vggface.git 'C:\Users\DELL\AppData\Local\Temp\pip-req-build-4z79fete'


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import os
import os.path
import PIL
import tensorflow as tf
import keras
import keras_vggface
import keras_vggface.utils

In [4]:
from keras_vggface.vggface import VGGFace
import mtcnn
import matplotlib.pyplot as plt
from keras.utils.data_utils import get_file
from keras.layers import Flatten, Dense, Input

In [5]:
vgg_face = VGGFace(model='resnet50', include_top=False,
                   input_shape=(224, 224, 3))
# print(vgg_face.summary())
print("Input: ", vgg_face.input)
print("Output: ", vgg_face.output)

94694792/94694792 [==============================] - 31s 0us/step
Input:  KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
Output:  KerasTensor(type_spec=TensorSpec(shape=(None, 1, 1, 2048), dtype=tf.float32, name=None), name='avg_pool/AvgPool:0', description="created by layer 'avg_pool'")


In [6]:
# import matplotlib.image as mpimg

# sample_img = mpimg.imread(
#     'data/train/TIN/20230413_161955.jpg').transpose(1, 0, 2)
# print(sample_img.shape)

# face_detector = mtcnn.MTCNN()
# face_roi = face_detector.detect_faces(sample_img)

In [7]:
# plt.imshow(sample_img)

In [8]:
# x1, y1, w, h = face_roi[0]['box']
# x2, y2 = x1 + w, y1 + h
# face = sample_img[y1:y2, x1:x2]
# print(face.shape)
# plt.imshow(face)

In [13]:
train_data = keras.utils.image_dataset_from_directory(
    'data/train', shuffle=True, batch_size=8, image_size=(224, 224))
test_data = keras.utils.image_dataset_from_directory(
    'data/test', shuffle=True, batch_size=8, image_size=(224, 224))

Found 120 files belonging to 3 classes.
Found 30 files belonging to 3 classes.


In [14]:
data_augmentation = keras.Sequential([
    keras.layers.Rescaling(1./255),
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.2),
])

number_of_classes = 3  # 2 person + 1 everything else

vgg_face.trainable = False  # freeze the weights
last_layer = vgg_face.get_layer('avg_pool').output

inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)  # randomly rotate/flip img
x = vgg_face(inputs)
x = Flatten(name='flatten')(x)
out = Dense(number_of_classes, activation='softmax', name='classifier')(x)
custom_model = keras.Model(inputs, out)

custom_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vggface_resnet50 (Functiona  (None, 1, 1, 2048)       23561152  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 classifier (Dense)          (None, 3)                 6147      
                                                                 
Total params: 23,567,299
Trainable params: 6,147
Non-trainable params: 23,561,152
_________________________________________________________________


In [15]:
custom_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
history = custom_model.fit(train_data, epochs=20)

Epoch 1/20
15/15 [==============================] - 20s 929ms/step - loss: 1.1419 - accuracy: 0.4417
Epoch 2/20
15/15 [==============================] - 16s 908ms/step - loss: 0.8784 - accuracy: 0.5083
Epoch 3/20
15/15 [==============================] - 15s 832ms/step - loss: 0.7226 - accuracy: 0.5667
Epoch 4/20
15/15 [==============================] - 14s 859ms/step - loss: 0.6123 - accuracy: 0.6000
Epoch 5/20
15/15 [==============================] - 17s 957ms/step - loss: 0.5326 - accuracy: 0.6750
Epoch 6/20
15/15 [==============================] - 22s 1s/step - loss: 0.4721 - accuracy: 0.7500
Epoch 7/20
15/15 [==============================] - 16s 948ms/step - loss: 0.4263 - accuracy: 0.8333
Epoch 8/20
15/15 [==============================] - 14s 834ms/step - loss: 0.3916 - accuracy: 0.8583
Epoch 9/20
15/15 [==============================] - 16s 977ms/step - loss: 0.3630 - accuracy: 0.8750
Epoch 10/20
15/15 [==============================] - 15s 897ms/step - loss: 0.3386 - accuracy:

In [16]:
print(custom_model.metrics_names)
custom_model.evaluate(test_data, batch_size=5)

['loss', 'accuracy']
4/4 [==============================] - 4s 549ms/step - loss: 0.4425 - accuracy: 0.8333


[0.4424825608730316, 0.8333333134651184]

In [17]:
custom_model.predict(test_data)

4/4 [==============================] - 4s 552ms/step


array([[1.5552082e-02, 8.1077907e-03, 9.7634012e-01],
       [9.8686856e-01, 1.1792519e-02, 1.3389366e-03],
       [3.0137282e-02, 1.5949494e-01, 8.1036776e-01],
       [2.4964905e-01, 3.1046090e-01, 4.3989003e-01],
       [8.7679528e-02, 5.1097202e-01, 4.0134847e-01],
       [5.7577077e-02, 3.1029579e-01, 6.3212717e-01],
       [9.9459463e-01, 5.4041711e-03, 1.2327425e-06],
       [1.6130295e-01, 4.3257701e-01, 4.0612006e-01],
       [4.4193543e-02, 4.0332065e-03, 9.5177329e-01],
       [1.0688779e-01, 2.3749147e-01, 6.5562075e-01],
       [9.9970716e-01, 2.9163179e-04, 1.1858406e-06],
       [9.8339766e-01, 1.6601879e-02, 4.1590576e-07],
       [1.0893076e-01, 5.7404572e-01, 3.1702355e-01],
       [1.5337671e-01, 4.6263152e-01, 3.8399175e-01],
       [7.7472269e-03, 9.9224055e-01, 1.2233077e-05],
       [9.9969482e-01, 3.0521682e-04, 2.2418046e-08],
       [5.0371099e-02, 1.0524641e-01, 8.4438258e-01],
       [9.9994731e-01, 5.1783849e-05, 9.5612199e-07],
       [1.0357729e-01, 1.096

In [18]:
custom_model.save('./model/model.h5')

In [19]:
import tensorflow as tf
model = tf.keras.models.load_model('./model/model.h5')

In [20]:
from PIL import Image
import numpy as np

img = Image.open('./data/test/TIN/20230413_162149.jpg')
img = img.resize((224, 224))
img = np.array(img)
img = img / 255.0
img = np.expand_dims(img, axis=0)

model.predict(img)

1/1 [==============================] - 1s 1s/step


array([[0.10695641, 0.09431466, 0.7987289 ]], dtype=float32)